# ADT Tutorial - multi device time synchronization
This tutorial shows how to obtain synchronized data from multiple devices, which share an aligned timecode time, but NOT device capture time. Note that this example requires at least two devices' data in the specified sequence. **Note that you need to download the `Apartment_release_multiskeleton_party_seq114` sequence for this tutorial**. 

### Notebook stuck?
Note that because of Jupyter and Plotly issues, sometimes the code may stuck at visualization. We recommend **restart the kernels** and try again to see if the issue is resolved.

In [ ]:
import numpy as np
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.graph_objects as go
import plotly.offline as pyo
import random

from projectaria_tools.core.stream_id import StreamId

from projectaria_tools.projects.adt import (
   AriaDigitalTwinDataProvider,
   AriaDigitalTwinDataPathsProvider,
   bbox3d_to_line_coordinates,
   utils as adt_utils,
)

### Download a multi-skeleton sequence

**sequence_path**: Enter the path to where the dataset has been downloaded to. The **Apartment_release_multiskeleton_party_seq114** should have been downloaded using the download script by running: 
```
${PROJECT_DIR}/adt_benchmark_dataset_downloader -c ${PATH_TO_CDN_LIST_FILE} -o ${OUTPUT_PATH} -d 0 -l Apartment_release_multiskeleton_party_seq114
```

In [ ]:
sequence_path = os.path.expanduser('~') + "/Documents/projectaria_tools_adt_data/Apartment_release_multiskeleton_party_seq114"
print('sequence path: ', sequence_path)

### Get all devices for a sequences 

ADT has sequences of multi-person activities. Each person is identified by the Aria serial number they wear. Use the AriaDigitalTwinDataPathsProvider to get all person ids, a.k.a Aria serial numbers 


In [ ]:
paths_provider = AriaDigitalTwinDataPathsProvider(sequence_path)
all_device_serials = paths_provider.get_device_serial_numbers()
print("device serials: ", all_device_serials)

### Load data from devices

For demonstration purpose, we only load two device's data from the sequence. 

In [ ]:
if len(all_device_serials) < 2:
    print("ERROR: need at least 2 valid devices")
    sys.exit(1)

data_providers=[]
for device_id, device_serial in enumerate(all_device_serials[:2]):
    print(f"device number - {device_id} : {device_serial}")
    current_data_paths = paths_provider.get_datapaths_by_device_num(device_id)
    print(f"loading ground truth data for device {device_id}...")
    data_providers.append(AriaDigitalTwinDataProvider(current_data_paths))

### Pick a sample timestamp from device_0

In [ ]:
stream_id = StreamId("214-1") # use rgb camera for testing
sample_frame_id = 1312 # A hand picked frame from device_0. You may need to change this if you are using a different sequence

# obtain device time and corresponding timecode time of this frame for device_0
device_time_0 = data_providers[0].get_aria_device_capture_timestamps_ns(stream_id)[sample_frame_id]
timecode_time_0 = data_providers[0].get_timecode_from_device_time_ns(device_time_0)

### Draw a dynamic object from device_0
Here we pick a dynamic object, and draw its 3D bounding box at `t = device_time_0` from device_0's time domain. 

In [ ]:
cup_instance_id = 4433484210031167  # A hand-picked object for this dataset (Black ceramic cup). You may need to change this if you are using a different dataset
bboxes_3d_0 = data_providers[0].get_object_3d_boundingboxes_by_timestamp_ns(device_time_0).data()
# check that the cup is within this frame
assert cup_instance_id in bboxes_3d_0, f"Device_0 3d bounding boxes from time {device_time_0} does not contain the black cup!"
cup_bbox_0 = bboxes_3d_0[cup_instance_id]

# transform axes-aligned bounding box to scene coordinate and visualize
aabb_coords = bbox3d_to_line_coordinates(cup_bbox_0.aabb)
obb_device_0 = np.zeros(shape=(len(aabb_coords), 3))
traces = []
for i in range(0, len(aabb_coords)):
    aabb_pt = aabb_coords[i]
    aabb_pt_homo = np.append(aabb_pt, [1])
    obb_pt = (cup_bbox_0.transform_scene_object.to_matrix()@aabb_pt_homo)[0:3]
    obb_device_0[i] = obb_pt

traces.append(go.Scatter3d(
        x=obb_device_0[:, 0],
        y=obb_device_0[:, 1],
        z=obb_device_0[:, 2],
        name="Cup from device_0",
        mode="lines",
        line=dict(color=[[0, 0, 255]]*len(obb_device_0), width=5),
))

pyo.init_notebook_mode()
layout = go.Layout(scene=dict(dragmode='orbit', aspectmode='data', xaxis_visible=False, yaxis_visible=False,zaxis_visible=False))
figure = go.Figure(data=traces, layout=layout)
pyo.iplot(figure)


### Query device timestamp from device_1 using synchronized timecode time
Timecode data from multiple devices should be aligned, therefore we can retrieve raw data according to aligned timecode timestamps. In this example, we can query device_1's device time by going: 
`device_time_from_device_0 -> timecode_time -> device_time_from_device_1`

Note that there are **interpolation** involved in both `get_device_time_from_timecode_ns()` and `get_timecode_from_device_time_ns()` functions, therefore there could be a **precision issue that results in a small numerical difference** when you perform a round trip conversion of `device_t -> timecode_t -> device_t`. 

In [ ]:
device_time_1 = data_providers[1].get_device_time_from_timecode_ns(timecode_time_0)

# Print some explanations for time domain conversion
timecode_time_1 = data_providers[1].get_timecode_from_device_time_ns(device_time_1)
print(f"We picked frame-{sample_frame_id} from device_0, which has device_time of {device_time_0}. \n")
print(f"Then we used `.get_timecode_from_device_time_ns()` function to obtain "
      f"the timecode time from device_0, which is {timecode_time_0}\n")
print(f"Because the timecode is aligned between device_0 and device_1, therefore "
     f"by calling `.get_device_time_from_timecode_ns({timecode_time_0})`, we can obtain device_time of {device_time_1} in device_1"
      f", which should be the timestamp that are actually aligned with frame-{sample_frame_id} in device_0 \n")
print(f"For validation, if we further convert device_1's device time {device_time_1} to timecode, "
      f"we get {timecode_time_1}, which should be very close to {timecode_time_0} "
      f"despite some numerical difference: {timecode_time_1 - timecode_time_0}\n")
print("Note that all units in timestamps are in nanoseconds (ns)")


### Draw the same dynamic object from device_1
Here we find the same dynamic object in device_1's data,  and draw its 3D bounding box at `t = device_time_1` from device_1's time domain. 
They should be aligned despite some difference in frame synchronization (difference up to 66ms)

In [ ]:
bboxes_3d_1 = data_providers[1].get_object_3d_boundingboxes_by_timestamp_ns(device_time_1).data()
# check that the cup is within this frame
assert cup_instance_id in bboxes_3d_1, f"Device_1 3d bounding boxes from time {device_time_1} does not contain the black cup!"
cup_bbox_1 = bboxes_3d_1[cup_instance_id]

# transform axes-aligned bounding box to scene coordinate and visualize
aabb_coords = bbox3d_to_line_coordinates(cup_bbox_1.aabb)
obb_device_1 = np.zeros(shape=(len(aabb_coords), 3))
for i in range(0, len(aabb_coords)):
    aabb_pt = aabb_coords[i]
    aabb_pt_homo = np.append(aabb_pt, [1])
    obb_pt = (cup_bbox_1.transform_scene_object.to_matrix()@aabb_pt_homo)[0:3]
    obb_device_1[i] = obb_pt

# Attach more lines to traces for plotting
traces.append(go.Scatter3d(
        x=obb_device_1[:, 0],
        y=obb_device_1[:, 1],
        z=obb_device_1[:, 2],
        name="Cup from device_1",
        mode="lines",
        line=dict(color=[[255, 0, 0]]*len(obb_device_1), width=5),
))

pyo.init_notebook_mode()
layout = go.Layout(scene=dict(dragmode='orbit', aspectmode='data', xaxis_visible=False, yaxis_visible=False,zaxis_visible=False))
figure = go.Figure(data=traces, layout=layout)
pyo.iplot(figure)

# Print the absolute difference between the object bounding boxes from device_0 and device_1
difference_sum = 0
for i in range(len(obb_device_0)):
    difference_sum += np.linalg.norm(obb_device_1[i] - obb_device_0[i])

print(f"the average bbox corner difference is {difference_sum / len(obb_device_0) * 1000} mm")
